## Build a chatbot using Gemini + Gradio

In [ ]:
# The library and API Key is added
import os
import google.generativeai as genai
import gradio as gr
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
    
model = genai.GenerativeModel('gemini-2.0-flash')
genai.configure(api_key=google_api_key)

### Let's test Gemini

In [ ]:
question = 'What is the capital of Argentina?'
response = model.generate_content(question)
response.resolve()
print(response.candidates[0].content.parts[0].text)

In [ ]:
chat = model.start_chat(history=[])
countries = ['Chile', 'Bolivia']

for country in countries:
    message = f'What is the capital of {country}?'
    chat.send_message(message)

chat.history

## Simple Chatbot using Gemini + Gradio

In [ ]:
SYSTEM_PROMPT="You are an helpful assistant. Answer the questions as best as you can."
chat = model.start_chat(history=[
    {"role": "user", "parts": [{"text": SYSTEM_PROMPT}]}
])

In [ ]:
# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for user_msg, bot_msg in history:
        new_history.append({"role": "user", "parts": [{"text": user_msg}]})
        new_history.append({"role": "model", "parts": [{"text": bot_msg}]})
    return new_history

In [ ]:
# Chat response generator
def response(message: str, history: list):
    try:
        # Sync Gemini chat history with Gradio history
        chat.history = [{"role": "user", "parts": [{"text": SYSTEM_PROMPT}]}] + transform_history(history)

        # Send user message
        gemini_response = chat.send_message(message)
        gemini_response.resolve()

        # Yield response character-by-character
        for i in range(len(gemini_response.text)):
            yield gemini_response.text[:i+1]

    except Exception as e:
        yield f"⚠️ Error: {str(e)}"

In [ ]:
# Launch Gradio ChatInterface
gr.ChatInterface(
    fn=response,
    title='Simple ChatBot with Gemini',
    description='This is a simple chatbot using Gemini. Ask me anything!',
    examples=[
        ['What is the capital of India?', 'New Delhi'],
        ['What is the name of our planet?', 'Earth']
    ],
    type="messages",
    textbox=gr.Textbox(placeholder="Question to Gemini")
).launch(debug=True)